In [1]:
import requests
import nba_api
import pandas as pd
import numpy as np
from datetime import date
from nba_api.stats.endpoints import (playbyplayv2,playbyplay, leaguehustlestatsplayer,
                                     leaguedashptstats,leaguestandings,
                                     playerdashptreb,leaguedashplayerbiostats)
import time

In [2]:
pbp2022=pd.read_csv(filepath_or_buffer="PBP Data/2021-22_pbp.csv")

# The Spark Plug Award (sponsored by Lt. Surge, presented by American Express CEO Stephen J Squeri)
Most charges drawn per 36 minutes (minimum 70% of games played), credit to morron88 for the idea to separate charges & loose balls in 2020

In [3]:
compact_standings=leaguestandings.LeagueStandings(league_id='00',
                                             season="2021-22",season_type="Regular Season").\
standings.get_data_frame()[['TeamID','TeamName','WINS','LOSSES']]
compact_standings['TeamGP']=compact_standings.WINS+compact_standings.LOSSES

hustle=leaguehustlestatsplayer.LeagueHustleStatsPlayer(
    per_mode_time="PerGame",season="2021-22",
    season_type_all_star="Regular Season").hustle_stats_player.get_data_frame()

In [4]:
hustle_w_gp_qualify=hustle.merge(compact_standings,how='left',left_on='TEAM_ID',right_on='TeamID')
hustle_w_gp_qualify['G_PERCENT']=hustle_w_gp_qualify.G/hustle_w_gp_qualify.TeamGP
hustle_70percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.7').copy()
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']]=\
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].\
div(hustle_70percent_gp.MIN,axis=0).multiply(36,axis=0)

In [5]:
hustle_70percent_gp.nlargest(5,columns='CHARGES_DRAWN',keep='all')[['PLAYER_NAME','CHARGES_DRAWN']]

,PLAYER_NAME,CHARGES_DRAWN
37,Blake Griffin,0.978035
192,Garrison Mathews,0.577099
344,Kevin Love,0.525664
459,Patrick Beverley,0.492187
422,Moritz Wagner,0.481690


# The Most Loose Balls Recovered Award (sponsored by Hungry Hungry Hippos, presented by Dennis Rodman & [Nene’s doctor](https://www.espn.com/nba/news/story?id=3197423))

Per 36 minutes, minimum 70% of games played

In [6]:
hustle_70percent_gp.nlargest(5,columns='LOOSE_BALLS_RECOVERED',keep='all')[['PLAYER_NAME','LOOSE_BALLS_RECOVERED']]

,PLAYER_NAME,LOOSE_BALLS_RECOVERED
210,Hamidou Diallo,1.364384
292,John Konchar,1.362712
484,Robert Williams III,1.350000
322,Jusuf Nurkic,1.276596
259,Jarred Vanderbilt,1.270588


# The Plexiglass Award

most deflections per 36 minutes, minimum 70% of games played

In [7]:
hustle_70percent_gp.nlargest(5,columns='DEFLECTIONS',keep='all')[['PLAYER_NAME','DEFLECTIONS']]

,PLAYER_NAME,DEFLECTIONS
195,Gary Payton II,5.306358
403,Matisse Thybulle,5.273438
128,De'Anthony Melton,4.656000
299,Jordan McLaughlin,4.614085
137,Dejounte Murray,4.148276


# The Wes Unseld Memorial Brick Wall Award

most points generated by screen assists per 36 minutes, minimum 70% of games played

In [8]:
hustle_70percent_gp.nlargest(5,columns='SCREEN_AST_PTS',keep='all')[['PLAYER_NAME','SCREEN_AST_PTS']]

,PLAYER_NAME,SCREEN_AST_PTS
492,Rudy Gobert,17.506542
521,Steven Adams,16.322727
244,Jakob Poeltl,15.402062
322,Jusuf Nurkic,15.063830
212,Hassan Whiteside,14.877348


# The No Fly Zone Award (presented by Dikembe Mutumbo)*

most blocked dunks as the blocking player

In [9]:
misses=pbp2022[pbp2022.HOMEDESCRIPTION.str.startswith('MISS',na=False)| 
           pbp2022.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
missed_dunks=misses[misses.HOMEDESCRIPTION.str.contains('Dunk',na=False)|
                   misses.VISITORDESCRIPTION.str.contains('Dunk',na=False)]
blocked_dunks=missed_dunks[missed_dunks.HOMEDESCRIPTION.str.contains('BLOCK',na=False)|
                   missed_dunks.VISITORDESCRIPTION.str.contains('BLOCK',na=False)]
#as the blockee
blocked_dunks.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME,count
42,Darius Bazley,9
40,Daniel Gafford,7
92,Ivica Zubac,7
128,Karl-Anthony Towns,7
169,OG Anunoby,7
178,Precious Achiuwa,7


# The Rejected for Boarding Award (sponsored by United Airlines)*

most blocked dunks as the dunking player (credit to Legdrop\_soup for the idea and asw7412 for the sponsor)

In [10]:
#as the blocker
blocked_dunks.groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER3_NAME,count
94,Jaren Jackson Jr.,10
9,Andre Drummond,9
39,Daniel Gafford,9
96,Jarrett Allen,9
152,Myles Turner,8


# The “Oops, I Dunked It Again” Award (sponsored by Britney Spears, presented by Gary Payton & Shawn Kemp)*

Most prolific alley-oop duo (credit to lactardenthusiast for the idea)

In [11]:
#get alley oops
alley_oops=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Alley',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Alley',na=False)]
#remove missed alley oop attempts
made_alley_oops=alley_oops[~alley_oops.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~alley_oops.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
#remove alley oops that have missing 2nd player (self-alley oop or perhaps missing data?)
made_alley_oops_complete=made_alley_oops[~made_alley_oops.PLAYER2_NAME.isna()].copy()
made_alley_oops_complete['p1']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].min(axis=1)
made_alley_oops_complete['p2']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].max(axis=1)
made_alley_oops_complete.groupby(['p1','p2']).size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,p1,p2,count
210,Clint Capela,Trae Young,54
339,Dwight Powell,Luka Doncic,36
248,Darius Garland,Jarrett Allen,35
606,Mike Conley,Rudy Gobert,32
518,John Collins,Trae Young,31
594,Marcus Smart,Robert Williams III,31


# The “He Trick Y’All, Running Around, Doing Nothing” Award (sponsored by Russell Westbrook, presented by Tony Snell)*

Lowest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played)

In [12]:
hustle_50percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.5').copy()
#traditional defensive stats approximate by tracking
#(deflections ~ steals, contested shots ~ blocks, def reb ~ boxouts)
per_36_percent_ranks=hustle_50percent_gp[['CHARGES_DRAWN', 'CONTESTED_SHOTS', 'DEFLECTIONS', 
               'DEF_BOXOUTS','DEF_LOOSE_BALLS_RECOVERED']].\
div(hustle_50percent_gp.MIN,axis=0).multiply(36,axis=0).apply(lambda x: x.rank(pct=True)).add_suffix("_pct_rank")
per_36_percent_ranks["sum"]=per_36_percent_ranks.sum(axis=1)
hustle_50percent_gp_ranks=hustle_50percent_gp.merge(per_36_percent_ranks,how='left',left_index=True,right_index=True)
hustle_50percent_gp_ranks.nsmallest(n=5,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
584,Will Barton,0.156160,0.206304,0.252149,0.008596,0.126074,0.749284
530,Terrence Ross,0.156160,0.106017,0.292264,0.108883,0.117479,0.780802
164,Duane Washington Jr.,0.156160,0.340974,0.197708,0.189112,0.034384,0.918338
22,Anfernee Simons,0.438395,0.177650,0.028653,0.014327,0.260745,0.919771
286,Joe Ingles,0.156160,0.051576,0.203438,0.481375,0.031519,0.924069


In [13]:
hustle_50percent_gp_ranks[hustle_50percent_gp_ranks['PLAYER_NAME']=='Patrick Beverley'].filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
459,Patrick Beverley,0.988539,0.097421,0.919771,0.054441,0.905444,2.965616


# The "Mr. Fantastic" Award (presented by Shane Battier, the No-Stats All-Star)*

Highest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played) (credit to memeticengineering for the idea)

In [14]:
hustle_50percent_gp_ranks.nlargest(n=5,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
533,Thaddeus Young,0.991404,0.819484,0.931232,0.862464,0.962751,4.567335
162,Draymond Green,0.888252,0.810888,0.885387,0.922636,0.919771,4.426934
133,DeMarcus Cousins,0.939828,0.851003,0.942693,0.911175,0.719198,4.363897
17,Alperen Sengun,0.945559,0.914040,0.896848,0.925501,0.315186,3.997135
309,Josh Okogie,0.914040,0.661891,0.954155,0.679083,0.785100,3.994269


# The Bowling Ball Award (sponsored by [Pete Weber](https://www.youtube.com/watch?v=gKQOXYB2cd8), presented by Glen "Big Baby" Davis)*

most charges committed (credit to Kdog122025 for the idea)

In [15]:
charges=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Charge',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Charge',na=False)]

charges.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME,count
113,Giannis Antetokounmpo,16
171,Jusuf Nurkic,14
167,Julius Randle,12
141,Jaren Jackson Jr.,10
149,Jerami Grant,10
189,Kyle Kuzma,10
196,Luguentz Dort,10
235,Pascal Siakam,10
271,Tobias Harris,10


# "The Good Ol' Hockey Game, is the Best Game You Can Name" Award (presented by Dominik Hasek)*

most goaltends committed (credit to Kdog122025 for the idea)

In [16]:
goaltends=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Goaltending',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('Goaltending',na=False)]
goaltends.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME,count
34,Clint Capela,13
8,Andre Drummond,11
143,Myles Turner,11
82,JaVale McGee,10
164,Richaun Holmes,10


# The Anti-Verticality Award (presented by Roy Hibbert)*

most 3-point shooting fouls committed (credit to watchingsongsDL, kingcobweb & An-Indian-In-The-NBA for the idea)

In [17]:
third_free_throws=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)|
                          pbp2022['VISITORDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)]

shooting_fouls=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('S.FOUL',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('S.FOUL',na=False)]

fouls_on_missed_threes=shooting_fouls.merge(right=third_free_throws,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

three_pointers=pbp2022[pbp2022['HOMEDESCRIPTION'].str.contains('3PT',na=False)|
        pbp2022['VISITORDESCRIPTION'].str.contains('3PT',na=False)]

made_threes=three_pointers[~three_pointers.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~three_pointers.VISITORDESCRIPTION.str.startswith('MISS',na=False)]

fouls_on_made_threes=shooting_fouls.merge(right=made_threes,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

fouls_on_threes=fouls_on_missed_threes.append(fouls_on_made_threes)
fouls_on_threes.groupby('PLAYER1_NAME_x').size().reset_index().rename(columns={0:'count'}).nlargest(5,columns='count',keep='all')

,PLAYER1_NAME_x,count
112,Garrett Temple,7
86,Desmond Bane,6
128,Isaac Okoro,6
261,Quentin Grimes,6
314,Tyrese Maxey,6


# The “Fine, I’ll Do It Myself” Award (sponsored by Thanos, presented by Allen Iverson)

Highest percentage of unassisted field goals, minimum 50% of games played (https://www.nba.com/stats/players/scoring/?sort=GP&dir=-1)

1. Luka Doncic (86%)
2. Chris Paul (84.1%)
3. Shai Gilgeous-Alexander (83.2%)
4. Trae Young (82.9%)
4. James Harden (80.5%)

# The “You Gotta Feed Me” Award (presented by Joey Chestnut & Marcin Gortat)

Highest percentage of assisted field goals, minimum 50% of games played

1. Davis Bertans (95.5%)
2. Wayne Ellington (95.4%)
3. Duncan Robinson & Ben McLemore (93.1%)
4. Mike Muscala & Garrison Mathews (92.9%)

# The “FUCK OUTTA HERE, I GOT THAT SHIT” Award

Lowest contested rebound percentage, minimum 50% of games played

In [18]:
rebounding=playerdashptreb.PlayerDashPtReb(team_id=0,player_id=0).overall_rebounding.get_data_frame()

games_percentages=hustle_w_gp_qualify.copy()[['PLAYER_ID','PLAYER_NAME','G_PERCENT']]

reb_w_gp_qualify=rebounding.merge(games_percentages,left_on='PLAYER_ID',right_on='PLAYER_ID').query('G_PERCENT >= 0.5')

reb_w_gp_qualify.nsmallest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
283,D.J. Augustin,0.065
160,Tyus Jones,0.072
318,Aaron Holiday,0.099
417,Isaiah Joe,0.100
126,Trae Young,0.102


In [19]:
reb_w_gp_qualify[reb_w_gp_qualify['PLAYER_NAME']=='Carmelo Anthony'][['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
274,Carmelo Anthony,0.313


alternatively: restricting to players > 6 foot 6 inches in height

In [20]:
player_bio=leaguedashplayerbiostats.LeagueDashPlayerBioStats().league_dash_player_bio_stats.get_data_frame()

above_66=player_bio.query('PLAYER_HEIGHT_INCHES > 6*12+6')

above_66.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nsmallest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
46,Furkan Korkmaz,0.129
41,Duncan Robinson,0.144
4,Amir Coffey,0.151
160,Ziaire Williams,0.155
14,Cam Reddish,0.162


# The "Where There's a Will, There's a Way" Award (presented by Dennis Rodman)

Highest contested rebound percentage, minimum 50% of games played

In [21]:
reb_w_gp_qualify.nlargest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
56,Mitchell Robinson,0.610
157,Jock Landale,0.587
153,Jakob Poeltl,0.584
248,Ivica Zubac,0.528
152,Steven Adams,0.527


alternatively: restricting to players < 6 foot 7 inches in height

In [22]:
below_67=player_bio.query('PLAYER_HEIGHT_INCHES < 6*12+7')
below_67.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nlargest(n=5,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
169,Thanasis Antetokounmpo,0.447
83,Ish Wainright,0.405
114,Kenrich Williams,0.391
161,Stanley Johnson,0.378
86,Jae'Sean Tate,0.375


# The Rotation Awards

**value depth over one solitary star. used basketball-reference's play-by-play position percentages as initial baseline for position groupings**

# The Best Guard Rotation Award (sponsored by Buckingham Palace)

East:
1. Heat (Lowry, Herro, Vincent)
2. Bucks (Jrue, George Hill, DiVincenzo)
3. Sixers (Harden, Maxey, Seth, Shake)
4. Bulls (Lonzo, Caruso, Coby, Ayo)
5. Boston (Smart, Schroder, Fast PP, Maximum Derrick)
6. Cavaliers (Garland, Rubio, Sexton, LeVert)
7. Raptors (FVV, GTJ, Malachi)
8. Nets (Harden, Seth, Patty Mills, [World B Flat](https://www.basketball-reference.com/players/i/irvinky01.html), Jevon Carter)
9. Hornets (LaMelo, Scary Terry, Ish Smith)
10. Hawks (Trae, Delon, Lou Will)
11. Wizards (Beal, Dinwiddie, Neto, Aaron Holiday)
12. Knicks (Quickley, Kemba, Grimes, Rose)
13. Pacers (Duarte, Brogdon, Haliburton, Duane Washington, TJ McConnell)
14. Pistons (Cory Jo, Killian, Frank Jackson, Saben Lee)
15. Magic (Cole Anthony, Suggs, Hampton, Fultz)

West:

1. Suns (CP3, Cam Payne, Elf)
2. Grizzlies (Ja, Melton, Tyus Jones)
3. Warriors (Steph, Poole, GPII)
4. Jazz (Spida, Conley, Trent Forrest)
5. Mavericks (Brunson, Luka, Frankie Smokes, Trey Burke, Dinwiddie)
6. Nuggets (Monte Morris, Campazzo, Bones)
7. Wolves (D'Angelo Russell, Malik Beasley, Pat Bev, Jordan McLaughlin)
8. Clippers (Big Government, Kennard, Bledsoe, Norm Powell)
9. Lakers (Westbrook, Avery Bradley)
10. Pelicans (Devonte' Graham, McCollum, Garrett Temple, Jose Alvarado, Kira Lewis)
11. Spurs (Dejounte, Maximum Derrick, Tre Jones, Bryn Forbes)
12. Trail Blazers (Simons, Dame, DSJ)
13. Kings (Fox, Hali, Davion Mitchell)
14. Thunder (SGA, Tre Mann, Ty Jerome, Theo Maledon)
15. Rockets (Jalen Green, KPJ, Armoni Brooks, DJ Augustin)

# The Best Wing Rotation Award (co-sponsored by Lou Williams and Magic City)

East:
1. Heat (Butler, Strus, [Sheen](https://twitter.com/thenextonepod/status/1307500403924369409), Caleb Martin)
2. Bucks (Middleton, Connaughton, Jordan Nwora, Hood, Wesley Matthews, [Ted Cruz](https://www.cbssports.com/college-basketball/news/look-ted-cruz-hilariously-embraces-grayson-allen-comparison-on-twitter/))
3. Sixers (Niang, Danny Green, Korkmaz, Thybulle)
4. Bulls (LaVine, Javonte Green, TBJ, DJJ)
5. Boston (Jay^2, Josh Richardson, Langford, Nesmith)
6. Cavaliers (Okoro, Cedi, Lamar Stevens, Dylan Windler)
7. Raptors (Scottie, Anunoby, Yuta, Svi, Dalano)
8. Nets (Bembry, James Johnson, Joe Harris, Bruce Brown, Cam Thomas)
9. Hornets (Hayward, Jalen McDaniels, Oubre, Cody Martin)
10. Hawks (Huerter, Bogdan^2, Hunter, Reddish, TLC)
11. Wizards (KCP, Corey Kispert, Deni Avdija, Hachimura)
12. Knicks (RJ, Fournier, Burks)
13. Pacers (Justin Holiday, Oshae, LeVert, Lamb, Lance, Buddy Hield)
14. Pistons (Saddiq Bey, Josh Jackson, Cade, Hamidou, McGruder)
15. Magic (Franz, Okeke, Gary Harris, Terrence Ross)

West:

1. Suns (Book, Bridges, Waluigi, Crowder, Torrey Craig)
2. Grizzlies (Bane, Brandon Clarke, Konchar, Ziaire Williams, Dillon Brooks, Xavier Tillman)
3. Warriors (Wiggins, JTA, Iggy, Klay, Damion Lee)
4. Jazz (Royce O'Neale, Jordan Clarkson, Bojan, Ingles)
5. Mavericks (DFS, Bullock, THJ, Josh Green, Sterling Brown)
6. Nuggets (Aaron Gordon, JaMychal, MPJ, Will Barton, Austin Rivers)
7. Wolves (Jaden McDaniels, Taurean Prince, Anthony Edwards, Jaylen Nowell, Okogie)
8. Clippers (Terance Mann, Marcus Morris, PG13, Covington, Amir Coffey, Brandon Boston)
9. Lakers (LeBron, Ariza, Monk, THT, Reaves, Ellington, Bazemore)
10. Pelicans (Josh Hart, NAW, Herb Jones, Brandon Ingram, Trey Murphy)
11. Spurs (Big Body, Keita Bates-Diop, Vassell, Lonnie Walker, Primo)
12. Trail Blazers (Norm Powell, Nassir Little, CJ Elleby, McCollum, McLemore, Tony Snell, Josh Hart)
13. Kings (Hield, Terence Davis, Justin Holiday, DiVincenzo, Jeremy Lamb, Harrison Barnes, Harkless)
14. Thunder (Giddey, Aaron Wiggins, DORT, Bazley, Kenrich)
15. Rockets (Jae'Sean Tate, Kenyon Martin Jr, Nwaba, Eric Gordon, Garry Bird, Josh Christopher)

# The Best Big Rotation Award (jointly sponsored by Tom Hanks, Cadbury and Sex and the City)

East:

1. Heat (Bam, Yurt7, Dedmon, PJ Tucker)
2. Bucks (Giannis, Bobby Portis, Mamulekashvili)
3. Sixers (Embiid, Tobi, Drummond, Bball Paul)
4. Bulls (Deebo, Vooch, Tony Bradley)
5. Boston (Horford, Timelord, Grant Williams, Freedom)
6. Cavaliers (Allen, Mobley, Markkanen, Love)
7. Raptors (Siakam, ~~Pascal~~, Achiuwa, Khem Birch, Boucher)
8. Nets (KD, LMA, Blake Griffin, Claxton, Drummond)
9. Hornets (Miles Bridges, Mason Plumlee, Montrezl Harrell, PJ Washington)
10. Hawks (Capela, John Collins, Gallinari, Okongwu, Dieng)
11. Wizards (Kuzma, Montrezl, Gafford, Thomas Bryant, Porzingis)
12. Knicks (Randle, Mitch Rob, Obi, Taj Gibson, Nerlens)
13. Pacers (Sabonis, Myles Turner, Torrey Craig, Goga, IJax, Jalen Smith)
14. Magic (WCJ, Bamba, Mo Wagner)
15. Pistons (Beef Stew, Jerami Grant, Lyles, Olynyk, Garza, Bagley)

West:

1. Suns (Cam Johnson, Ayton, JaVale, Jalen Smith, Bismack)
2. Grizzlies (JJJ, Steven Adams, Kyle Anderson, Killian Tillie)
3. Warriors (Looney, OPJ, Dray, Bjelica, Kuminga)
4. Jazz (Gobert, Whiteside, Rudy Gay, Paschall)
5. Mavericks (Dwight Powell, Kleber, Porzingis, Bertans)
6. Nuggets (Jokic, Jeff Green, Nnaji)
7. Wolves (KAT, Vanderbilt, Naz Reid)
8. Clippers (Zubac, Batum, Hartenstein, Ibaka, Winslow)
9. Lakers (Carmelo, Day-to-Dayvis, Dwight, Stanley Johnson, DeAndre Jordan)
10. Pelicans (Valanciunas, Jaxson Hayes, Willy Hernangomez)
11. Spurs (Poeltl, Doug McDermott, Eubanks, Landale, Thad Young, Zach Collins)
12. Trail Blazers (Nurk, Covington, Larry Nance Jr, Trendon Watford)
13. Kings (Richaun, Sabonis, Metu, Bagley, Len, Damian Jones, Tristan Thompson)
14. Thunder (JRE, Poku, Favors, Muscala, Roby)
15. Rockets (Christian Wood, Alperen, Theis)